In [6]:
import numpy as np
import matplotlib.pyplot as plt
import math
import imageio
import os
import csv

Viscek:
Stated briefly as rules, and in $\textbf{order of decreasing precedence}$, the behaviors that lead to simulated flocking are:
1. Collision Avoidance: avoid collisions with nearby
flockmates
2. Velocity Matching: attempt to match velocity with nearby
flockmates
3. Flock Centering: attempt to stay close to nearby flockmates 

In [18]:
class Model:
    def __init__(self, dt = 1, density = 1, maxtime = 100, radius = 1, L = 10, noise = 0.05, phenotype = None, volume = 0.1):
        self.dt = dt 
        self.density = density
        self.num_agents = int(L**2 * density) 
        self.maxtime = maxtime
        self.r = radius # sam fix, put as an input of agents 
        self.curr_time = 0
        self.L = L
        self.agents = []
        self.volume = volume
        
        for i in range(self.num_agents):
            x = np.random.uniform(0,L,2)
            v = np.random.uniform(-1/2,1/2,2)
            v = v/np.linalg.norm(v)

            if phenotype == "Current+Centre_dt":
                p = {"Current":1 + self.dt, "Align" :0, "Centre":  self.dt} 
                            
            elif phenotype == "Centre+Align_dt":
                p = {"Current":self.dt, "Align" : self.dt, "Centre": 1 + self.dt}
                
            elif phenotype == "Current+Align_dt":
                p = {"Current":1 + self.dt, "Align" : self.dt, "Centre": 0}
                
            elif phenotype == "Current+Align_dt+Centre_dt":
                p = {"Current":1 + self.dt, "Align" : self.dt, "Centre": self.dt}
                            
            elif phenotype == "Viscek":
                p = {"Current":0, "Align" : 1, "Centre": 0} 
            else:
                p = {"Current":np.random.uniform(0,1), "Align" : np.random.uniform(0,1), "Centre":  np.random.uniform(0,1)}

            self.agents.append(Agent(pos = x, vel = v, parameters = p))
        
    def run(self):
        while self.curr_time < self.maxtime:
            self.step()
            self.curr_time += self.dt
    
    def step(self):
        positions = [x.pos[-1] for x in self.agents]
        velocities = [x.vel[-1] for x in self.agents]
        
        for a in self.agents:
                        
            nearby_pos = []
            nearby_vel = []
            
            for i, x in enumerate(positions):
                
                vector = np.remainder(x - a.pos[-1] + self.L/2, self.L) - self.L/2
        
                if np.linalg.norm(vector) < self.r:
                    nearby_pos.append(np.array(vector))
                    nearby_vel.append(velocities[i]-a.vel[-1])

            #nearby_pos = [x for x in positions if math.dist(x,a.pos[-1])<self.r]
            
            #nearby_vel = [v for i,v in enumerate(velocities) if math.dist(positions[i],a.pos[-1])< self.r]
            
            a.update_pos(self.dt, nearby_pos,nearby_vel)
            
            a.pos[-1] = a.pos[-1] % self.L # add other BC later
    
    def quiver_plot(self,i = -1, animate = False, name = None):
        positions = [x.pos[i] for x in self.agents]
        velocities = [x.vel[i] for x in self.agents]
        xs = [x[0] for x in positions]
        ys = [y[1] for y in positions]
        us = [u[0] for u in velocities]
        vs = [v[1] for v in velocities]
        plt.figure()
        plt.quiver(xs,ys,us,vs)
        plt.xlim([0,self.L])
        plt.ylim([0,self.L])
        if name:
            string = name + r", $\rho$ =" + str(self.density) + ", R =" + str(self.r)
            plt.title(string)
        if animate:
            plt.savefig('data/' + str(i)+'.png')
 
            plt.close()
        else:
            plt.show()
        
    def animate(self, name = "Gif"):
        entries = os.listdir('data/')
        for filename in entries:
            os.remove('data/' + filename)
        for i in range(len(self.agents[0].pos)):
            self.quiver_plot(i, True, name)
        entries = os.listdir('data/')
        entries = [int(x[:-4]) for x in entries]
        entries.sort()
        with imageio.get_writer(name + ".gif", mode='I') as writer:
            for i, filename in enumerate(entries):
                if i == 0:
                    for j in range(4):
                        image = imageio.imread('data/' + str(filename) + '.png')
                        writer.append_data(image)
                image = imageio.imread('data/' + str(filename) + '.png')
                writer.append_data(image)
                
    def blender_csv(self):
        # Create CSV with positions of each agent (only first agent for now)
        positions = self.agents[0].pos
        positions = [[x[0],x[1],0] for x in positions] # Add zero z-value for now
        np.savetxt('blender_scripts/positions.csv',positions, fmt = '%f', delimiter=",")
        
                
class Agent:
    def __init__(self, pos = [0,0], vel = [1,1], noise = 0.1, parameters = {"Current": 1, "Align" : 0, "Centre": 0}):
        self.pos = [pos]
        self.vel = [vel]
        self.noise = noise
        self.parameters = parameters 
        
    def update_pos(self,dt, nearby_pos, nearby_vel):
        
        current = self.vel[-1]
        
        if len(nearby_vel) > 0:
            
            align = sum(nearby_vel)/len(nearby_vel)

            centre = sum(nearby_pos)/len(nearby_pos)
      
            new_vel = self.parameters["Current"] * current + self.parameters["Align"] * align + self.parameters["Centre"] * centre + np.random.normal(0, self.noise, 2)
        
        else:
            new_vel = current
            
        if np.linalg.norm(new_vel) > 1:
            self.vel.append(new_vel/np.linalg.norm(new_vel))
        else:
            self.vel.append(new_vel)
        self.pos.append(self.pos[-1] + dt*self.vel[-1])
 

In [8]:
phenotypes = ["Centre+Align_dt"]
for phenotype in phenotypes:

    model = Model(dt = 0.1, maxtime = 50, noise = 0.01, phenotype = phenotype)
    model.run(), 
    model.animate(phenotype)

${\displaystyle \operatorname {distance} (P_{1},P_{2},(x_{0},y_{0}))={\frac {|(x_{2}-x_{1})(y_{1}-y_{0})-(x_{1}-x_{0})(y_{2}-y_{1})|}{\sqrt {(x_{2}-x_{1})^{2}+(y_{2}-y_{1})^{2}}}}.}$

# Create CSV of positions for blender (TEST)

In [19]:
phenotypes = ["Centre+Align_dt"]
for phenotype in phenotypes:

    model = Model(dt = 1, maxtime = 50, noise = 0.01, phenotype = phenotype)
    model.run(), 
    model.blender_csv()